In [782]:
import csv, re, math

In [783]:
def read_matrix_csv(file_path):
    with open(file_path, 'r') as f:
        reader = csv.reader(f)
        matrix = [list(map(int,row)) for row in reader]
    return matrix

In [784]:
def read_ink_file(file_path):
    orientation = 0
    ink_matrix = []
    with open(file_path,'r') as f:
        for line in f:
            if line.startswith("ORIENTATION:"):
                orientation = int(line.split(":")[1])
            if line.startswith("ROWDATA:"):
                row = list(map(int,line.strip().split(":")[1].split()))
                ink_matrix.append(row)
            if line.startswith("GOOD_DIE:"):
                good_die = int(line.split(":")[1])
            if line.startswith("ACTIVE_BAD_DIE:"):
                active_bad_die = int(line.split(":")[1])
            if line.startswith("INACTIVE_BAD_DIE:"):
                inactive_bad_die = int(line.split(":")[1])
            if line.startswith("DISTANCE:"):
                distance = int(line.split(":")[1])
            if line.startswith("DIESIZE:"):
                height, width = int(line.split(":")[1].split(",")[0]),int(line.split(":")[1].split(",")[1])
            # if line.startswith("INVALID_DIE:"):
            #     invalid_die = int(line.split(":")[1])
            if line.startswith("NAME:"):
                s = line.split(":")[1]
                testcase = re.search(r'\d+',s).group()
                
    return ink_matrix,orientation,good_die,active_bad_die,inactive_bad_die,distance,height, width, testcase


In [785]:
def rotate_matrix_90_counterclockwise(mat):

    transposed = list(zip(*mat))
    rotated = [list(row) for row in transposed[::-1]]

    return rotated

In [786]:
def align_orientation(ink_matrix, orientation):
    if orientation >= 90:
        mat =  rotate_matrix_90_counterclockwise(ink_matrix)
        if orientation >= 180:
            mat = rotate_matrix_90_counterclockwise(mat[:])
            if orientation >= 270:
                mat = rotate_matrix_90_counterclockwise(mat[:])
                if orientation >= 360:
                    mat = rotate_matrix_90_counterclockwise(mat[:])
        return mat
    return ink_matrix

In [ ]:
def align_center(ink_matrix, matrix):
    num_row_ink = len(ink_matrix)
    num_row_matrix = len(matrix)
    num_col_ink = len(ink_matrix[0])
    num_col_matrix = len(matrix[0])
    print(num_row_ink, num_row_matrix)
    diff1 = num_row_matrix - num_row_ink
    diff2 = num_col_matrix - num_col_ink
    print("diff: " ,diff1)
    mat =  []
    if diff1 == 0:
        return ink_matrix
    else:
        
        half1 = math.floor(diff1/2)
        half2 = 
        print("length: "+ str(len(matrix[0])))
        for i in range(half1):
            mat.append([0]*len(matrix[0]))
            print(mat[i])
        if diff1%2 != 0:
            mat.append([0]*len(matrix[0]))
        for i in range(num_row_ink):
            mat.append(ink_matrix[i])
        for i in range(half1):
            mat.append([0]*len(ink_matrix[0]))

    print(len(mat))
    return mat


In [788]:
def find_contaminated(matrix, new_ink_matrix, active_bad_die, inactive_bad_die, distance, height, width, contaminated):
    print("Rows: ",len(matrix))
    print("Colums: ",len(matrix[0]))
    print("ink Rows: ",len(new_ink_matrix))
    print("ink Colums: ",len(new_ink_matrix[0]))
    print(new_ink_matrix)
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            print("ink Colums: ",len(new_ink_matrix[j]))
            # print("Matrix len: ",len(matrix[0]))
            # print("Ink matrix len: ", len(new_ink_matrix[0]))
            print("Printing:")
            print(i,j)
            if new_ink_matrix[i][j] == active_bad_die:
                dies = 0
                #for right
                right = j + 1
                while dies < distance and right < len(new_ink_matrix[0]):
                    if new_ink_matrix[i][right] != inactive_bad_die:
                        new_ink_matrix[i][right] = contaminated # contaminated = 111
                    if i+1 < len(new_ink_matrix) and new_ink_matrix[i+1][right] != inactive_bad_die and new_ink_matrix[i+1][right] != 0:
                        new_ink_matrix[i+1][right] = contaminated
                    if i-1 >= 0 and new_ink_matrix[i-1][right] != inactive_bad_die and new_ink_matrix[i-1][right] != 0:
                        new_ink_matrix[i-1][right] = contaminated
                    dies +=width
                    right+=1

                # for left
                dies = 0
                left = j-1
                while dies < distance and left >= 0:
                    if new_ink_matrix[i][left] != inactive_bad_die:
                        new_ink_matrix[i][left] = contaminated
                    if i+1 < len(new_ink_matrix) and new_ink_matrix[i+1][left] != inactive_bad_die and new_ink_matrix[i+1][left] != 0:
                        new_ink_matrix[i+1][left] = contaminated
                    if i-1 >= 0 and new_ink_matrix[i-1][left] != inactive_bad_die and new_ink_matrix[i-1][left] != 0:
                        new_ink_matrix[i-1][left] = contaminated
                    dies+=width
                    left-=1

                # for top
                dies = 0
                top = i+1
                while dies < distance and top < len(new_ink_matrix):
                    if new_ink_matrix[top][j] != inactive_bad_die and new_ink_matrix[top][j] != 0:
                        print("Contaminated")
                        new_ink_matrix[top][j] = contaminated
                    else:
                        print("Inactive")
                    dies+=height
                    top+=1

                # for bottom
                dies = 0
                bottom = i-1
                while dies < distance and bottom >= 0:
                    if new_ink_matrix[bottom][j] != inactive_bad_die and new_ink_matrix[bottom][j] != 0:
                        new_ink_matrix[bottom][j] = contaminated
                    dies+=height
                    bottom-=1

            

In [789]:
def apply_ink_to_matrix(matrix, ink_matrix, good_die, active_bad_die, inactive_bad_die, contaminated):
    # output = []
    good_die_num, bad_die_num , contaminated_die_num = 0, 0, 0
    for i in range(len(matrix)):
        row = []
        for j in range(len(matrix[0])):
            # print(i,j)
            if matrix[i][j] == 0 or ink_matrix[i][j] == 0:
                # row.append("000")
                matrix[i][j] = "000"
            else:
                if ink_matrix[i][j] == good_die:
                    # row.append("00G")
                    matrix[i][j] = "00G"
                    good_die_num+=1
                if (ink_matrix[i][j] == active_bad_die) or (ink_matrix[i][j] == inactive_bad_die):
                    # row.append("00B")
                    if (ink_matrix[i][j] == active_bad_die) :
                        print("Active bad die")
                    if (ink_matrix[i][j] == inactive_bad_die) :
                        print("InActive bad die")
                    matrix[i][j] = "00B"
                    bad_die_num+=1
                if ink_matrix[i][j] == contaminated:
                    # row.append("00C")
                    matrix[i][j] = "00C"
                    contaminated_die_num+=1
                
        # output.append(row)
    return matrix, good_die_num, bad_die_num, contaminated_die_num

In [ ]:
def write_output(output_matrix, testcase, good_die_num, bad_die_num, contaminated_die_num):
    output_file = "M2T"+str(testcase)+'.txt'
    
    with open(output_file, 'w') as f:
        f.write("NO OF GOOD DIES: "+str(good_die_num)+'\n')
        f.write("NO OF BAD DIES: "+ str(bad_die_num)+'\n')
        f.write("NO OF CONTAMINATED DIES: "+ str(contaminated_die_num)+'\n')
        for row in output_matrix:
            f.write("ROWDATA:"+" ".join(row)+"\n")

In [791]:
contaminated_die = 111

matrix = read_matrix_csv('Students\Dataset\M2\T6\MATRIX.csv')
ink_matrix, orientation, good_die, active_bad_die, inactive_bad_die, distance, height, width, testcase = read_ink_file('Students\Dataset\M2\T6\INPUT.txt')
# print(distance)
# print(height)
# print(width)
# print(good_die)
print("Inactive bad die:" ,inactive_bad_die)
print("active bad die:" ,active_bad_die)
ink_matrix = align_orientation(ink_matrix, orientation)
ink_matrix = align_center(ink_matrix, matrix)
print("Ink matrix: ")
print(len(ink_matrix))
print(len(matrix))
print(len(ink_matrix[0]))
print(len(matrix[0]))
find_contaminated(matrix, ink_matrix, active_bad_die, inactive_bad_die, distance, height, width, contaminated_die)
output, good_die_num, bad_die_num, contaminated_die_num = apply_ink_to_matrix(matrix, ink_matrix,good_die,active_bad_die, inactive_bad_die, contaminated_die)
print(good_die_num)
print(bad_die_num)
print(contaminated_die_num)
write_output(output,testcase, good_die_num, bad_die_num, contaminated_die_num)


Inactive bad die: 3
active bad die: 6
36 40
diff:  4
length: 40
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
40
Ink matrix: 
40
40
40
40
Rows:  40
Colums:  40
ink Rows:  40
ink Colums:  40
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,

IndexError: list index out of range